**About** : This notebook is used to train detection models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
import os
import re
import cv2
import sys
import ast
import glob
import json
import yaml
import shutil
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from distutils.dir_util import copy_tree

warnings.filterwarnings("ignore", category=UserWarning)
pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [ ]:
from params import *
from util.plots import *
from inference.yolox import retrieve_yolox_model, predict, YoloXWrapper
from inference.utils import get_transfos, InferenceDataset
from util.metrics import *

from post_process.similarity import extract_similarities
from post_process.ticks import restrict_on_line, assign
from post_process.in_graph import post_process_preds

### Load data

In [ ]:
df = pd.read_csv('../input/df_train.csv')
df_text = pd.read_csv('../input/texts.csv')
df_target = pd.read_csv('../input/y_train.csv')
df_elt = pd.read_csv('../input/elements.csv')

In [ ]:
df = df[~df['id'].isin(ANOMALIES)].reset_index(drop=True)

In [ ]:
df_split = pd.read_csv('../input/df_split.csv')
df = df.merge(df_split)

In [ ]:
CLASSES = [
#     "dot",
#     "line",
    "scatter",
]

df = df[df['chart-type'].isin(CLASSES)].reset_index(drop=True)

### Model

In [ ]:
class ConfigMarker:
    selected_model = "yolo"
    bbox_format = "yolo"
    pred_format = "pascal_voc"

    weights = "/workspace/kaggle_benetech/logs/yolov7x-w6-v8.2/weights/best.pt"

    version = "v5"
    labels = ['chart', 'text', 'tick', 'point']
    
    size = (640, 640)

    
    conf_thresh = [0.1, 0.4, 0.2, 0.1]
    iou_thresh = [0.5, 0.25, 0.25, 0.1]


    max_per_img = 500
    min_per_img = 1
    
    val_bs = 16
    device = "cuda"
    
config_marker = ConfigMarker
VERSION = config_marker.version

In [ ]:
# assert os.path.exists(config_marker.weights), "Weights do not exist"
# model_marker = retrieve_model(config_marker)

In [ ]:
class ConfigMarker:
    selected_model = "yolo"
    bbox_format = "yolo"
    pred_format = "pascal_voc"
    
    name = "benetech_1_m_1"
    
#     version = "v11_sim"
    version = "v13"
    labels = ["point"]

    size = (1024, 1024)

    # NMS
    conf_thresh = 0.4
    iou_thresh = 0.2
    max_per_img = 500
    min_per_img = 1
    
    val_bs = 1  # if size[0] > 1024 else 16
    device = "cuda"
    
config_marker = ConfigMarker

In [ ]:
model_marker = retrieve_yolox_model(config_marker.name)
model_marker = YoloXWrapper(model_marker, config_marker)

In [ ]:
# FOLDER = "../output/weights/det_1/"
# name = config_marker.weights.split('/')[-3]

# cp = torch.load(config_marker.weights)

# import yaml
# with open(FOLDER + name + '_cfg.yml', 'w') as outfile:
#     yaml.dump(cp['model'].yaml, outfile)
    
# from util.torch import save_model_weights
# torch.save(cp['model'].state_dict(), FOLDER + name + "_weights.pt")

# print('-> Saved config to', FOLDER + name + '_cfg.yml')
# print('-> Saved weight to', FOLDER + name + '_weights.pt')

### Data

In [ ]:
VERSION = "v13"

In [ ]:
# from pathlib import Path

# df = pd.DataFrame({"path": glob.glob('../input/dots/*')})
# df['id'] = df['path'].apply(lambda x: Path(x).stem)
# df['source'] = "extracted"
# df['chart-type'] = "dot"
# df['gt_path'] = ""


In [ ]:
# # df = pd.DataFrame({"path": glob.glob(f'../input/{VERSION}/images/*/*.jpg')})

# df = pd.DataFrame({"path": glob.glob(f'../input/{VERSION}/val2017/*.jpg')})
# df['id'] = df['path'].apply(lambda x: x[:-4].split('/')[-1])
# df['gt_path'] = f'../input/{VERSION}/labels/valid/' + df['id'] + '.txt'

In [ ]:
len(df)

In [ ]:
# df = pd.DataFrame({"path": glob.glob(f'../input/{VERSION}/train2017/*.jpg')})
# df['id'] = df['path'].apply(lambda x: x[:-4].split('/')[-1])
# df['gt_path'] = f'../input/{VERSION}/labels/train/' + df['id'] + '.txt'

In [ ]:
len(df)

### Main

In [ ]:
PLOT = True
DEBUG = False
SAVE = False

In [ ]:
ids = np.arange(len(df))

chunk_size = 100
chunks = [ids[i: i + chunk_size] for i in range(0, len(ids), chunk_size)]

In [ ]:
# if SAVE:
#     SAVE_FOLDER = f"../input/{VERSION}_sim/"
#     os.makedirs(SAVE_FOLDER, exist_ok=True)

#     os.makedirs(SAVE_FOLDER + "train2017/", exist_ok=True)
#     os.makedirs(SAVE_FOLDER + "val2017/", exist_ok=True)

#     os.makedirs(SAVE_FOLDER + "labels/", exist_ok=True)
#     _ = copy_tree(f"../input/{VERSION}/labels/train", f"../input/{VERSION}_sim/labels/train")
#     _ = copy_tree(f"../input/{VERSION}/labels/valid", f"../input/{VERSION}_sim/labels/valid")
#     _ = copy_tree(f"../input/{VERSION}/annotations", f"../input/{VERSION}_sim/annotations")

In [ ]:
%matplotlib inline

In [ ]:
for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    
    df_val = df.iloc[chunk].reset_index(drop=True)
    
    if DEBUG:
        df_val = df_val.head(1)

    transforms = get_transfos(size=config_marker.size)
    dataset = InferenceDataset(df_val, transforms)
    
    meter_marker, fts = predict(model_marker, dataset, config_marker, extract_fts=True)
    
    dataset = InferenceDataset(df_val, None)
    
    scores = []
    for idx in range(len(dataset)):
        img, gt, shape = dataset[idx]

        preds = meter_marker.preds[idx]
        preds.update_shape(shape)

        preds = [
            preds['pascal_voc'][meter_marker.labels[idx] == i]
            for i in range(len(config_marker.labels))
        ]

#         preds = post_process_preds(preds)

#     #     break
#         if PLOT:
#             plot_results(img, [[], [], [], preds[-1]], figsize=(12, 7))
# #     #     break
        try:
            sim_img = extract_similarities(fts[idx], preds, img, verbose=0)
        except Exception:
            print('Error extracting similarities')
            sim_img = np.zeros(img.shape)

        img_final = np.concatenate([
            img.mean(-1, keepdims=True),
            (1 - sim_img[:, :, :2]) * 255
        ],-1).astype(np.uint8)
        
        if SAVE:
            cv2.imwrite(
                re.sub(f'/{VERSION}/', f'/{VERSION}_sim/', df_val['path'][idx]),
                img_final
            )

        if PLOT or DEBUG or np.random.random() < 0.001:
            plt.figure(figsize=(15, 7))
            plt.subplot(1, 2, 1)
            plt.imshow(sim_img)
            plt.axis(False)
            plt.subplot(1, 2, 2)
            plt.imshow(img)
            plt.axis(False)
            plt.show()
            
#             plt.figure(figsize=(15, 7))
#             plt.imshow(img_final)
#             plt.show()

        if DEBUG:
            break
    if DEBUG:
        break

Done ! 